In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP
logSH = pd.read_csv(r'F:\data\20200812\logs_20200812_zt_88_03_day_88data\mdLog_SH_20200812_0849.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]
logSH1 = pd.read_csv(r'F:\data\20200812\full_20200812_zt_70_01_day_7079\mdLog_SH_20200812_1359.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]

# # 1. data format
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

new data source type: 
[25]
['08:49:30.000' '08:49:30.470' '08:49:31.370' ... '15:14:02.110'
 '15:14:02.000' '15:14:03.000']
['13:59:56' '13:59:57' '13:59:58' ... '15:19:46' '15:19:48' '15:19:49']


In [123]:
logSH1 = pd.read_csv(r'F:\data\20200813\full_20200813_zt_70_01_day_7079\mdLog_SH_20200813_1122.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]
logSH1["time"] = logSH1["time"].apply(lambda x: int(x.replace(':', "").replace('.', ""))) * 1000

In [6]:
logSH["time"] = logSH["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int(x.replace(':', "").replace('.', ""))) * 1000

display(logSH[(logSH["StockID"] == 600000) & (logSH["time"] >= 104000000) & (logSH["time"] <= 104500000) & (logSH['source'] == 23)])
display(logSH1[(logSH1["StockID"] == 600000) & (logSH1["time"] >= 104000000) & (logSH1["time"] <= 104500000)])

,clockAtArrival,sequenceNo,source,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
4737575,1597200025750325,105649521,23,600000,104002000,19924728,2.099857e+08,10.53,10.53,10.52,10.51,10.5,10.49,59200,49627,44100,288300,47400,10.54,10.55,10.56,10.57,10.58,87800,250529,109000,126200,287000,10.49
4740516,1597200028744357,105706822,23,600000,104005000,19925928,2.099984e+08,10.53,10.53,10.52,10.51,10.5,10.49,60000,50327,44100,288300,47400,10.54,10.55,10.56,10.57,10.58,34500,250529,109000,126200,287000,10.49
4743432,1597200031752017,105753110,23,600000,104008000,19926028,2.099994e+08,10.53,10.53,10.52,10.51,10.5,10.49,59900,50327,44100,288300,47400,10.54,10.55,10.56,10.57,10.58,34500,250529,109000,126200,287000,10.49
4746347,1597200034751188,105798113,23,600000,104011000,19927128,2.100110e+08,10.53,10.53,10.52,10.51,10.5,10.49,58800,110127,44100,288300,47400,10.54,10.55,10.56,10.57,10.58,35500,250529,109000,126200,287000,10.49
4749222,1597200037752107,105842859,23,600000,104014000,19928128,2.100215e+08,10.53,10.53,10.52,10.51,10.5,10.49,53000,110127,44100,288300,47400,10.54,10.55,10.56,10.57,10.58,35800,250329,109000,126200,287000,10.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5006818,1597200308322653,109742826,23,600000,104444000,20445855,2.154723e+08,10.53,10.53,10.52,10.51,10.5,10.49,39400,127200,301700,308100,43900,10.54,10.55,10.56,10.57,10.58,42973,253629,104600,125400,298000,10.49
5009649,1597200311319234,109784105,23,600000,104447000,20449255,2.155081e+08,10.53,10.53,10.52,10.51,10.5,10.49,36100,127200,301700,308100,43900,10.54,10.55,10.56,10.57,10.58,42773,253629,104600,125400,298000,10.49
5012292,1597200314265354,109826270,23,600000,104450000,20459655,2.156177e+08,10.53,10.53,10.52,10.51,10.5,10.49,34100,127200,301700,308100,43900,10.54,10.55,10.56,10.57,10.58,34673,213629,104600,125400,298000,10.49
5017875,1597200319939822,109906017,23,600000,104456000,20459655,2.156177e+08,10.53,10.53,10.52,10.51,10.5,10.49,34100,128700,301700,308100,43900,10.54,10.55,10.56,10.57,10.58,35473,213629,104600,125400,298000,10.49


,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice


In [3]:
logSH1['time'].max()

104714000

In [98]:
logSH[~logSH["StockID"].isin(in_dex) & (logSH['source'] == 23)]['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique()

array([1, 2, 6, 7], dtype=int64)

In [7]:
logSH1['time'].min()

135956000

In [8]:
in_dex = [16, 300, 852, 905]
data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH['source'] == 23) & (logSH['time'] >= 135956000)]
data2 = logSH1[~logSH1["StockID"].isin(in_dex)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", 'openPrice']
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]

# data1_1['openPrice'] = np.where(data1_1['cum_volume'] > 0, data1_1.groupby('StockID')['openPrice'].transform('max'), data1_1['cum_amount'])
for cols in ["cum_amount"]:
    data2_1[cols] = (data2_1[cols]*10).round(2) 
    data1_1[cols] = data1_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    display(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)
    print((n2-n1)/n1)

1667
1667
1544413
1544333
1620176
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange,time_y
0,10985372.0,1.597212e+15,194258385.0,23.0,601996,135956000.0,7900177,2.292667e+07,2.89,2.89,2.88,2.87,2.86,2.85,57952,195900,296000,206900,118600,2.90,2.91,2.92,2.93,2.94,125300,161100,523200,274200,141300,2.93,NaN,NaN,NaN,NaN,NaN,NaN
1,10985373.0,1.597212e+15,194258386.0,23.0,603035,135956000.0,30753970,4.551477e+08,15.38,15.37,15.36,15.35,15.33,15.32,5800,9800,1400,1500,4600,15.38,15.39,15.40,15.41,15.42,2500,19200,44700,31500,16000,14.56,NaN,NaN,NaN,NaN,NaN,NaN
3,10985376.0,1.597212e+15,194258389.0,23.0,601233,135956000.0,20933993,3.192298e+08,14.90,14.89,14.87,14.86,14.85,14.84,700,3200,9400,5800,5100,14.90,14.91,14.92,14.93,14.94,21100,25900,14800,10200,40600,15.15,NaN,NaN,NaN,NaN,NaN,NaN
4,10985377.0,1.597212e+15,194258390.0,23.0,600571,135956000.0,5914198,5.305818e+07,8.88,8.88,8.87,8.86,8.85,8.84,25800,40600,57100,38000,15700,8.89,8.90,8.91,8.92,8.93,3400,108100,29200,4400,2000,9.10,NaN,NaN,NaN,NaN,NaN,NaN
5,10985378.0,1.597212e+15,194258391.0,23.0,600818,135956000.0,2532542,2.992454e+07,11.73,11.69,11.68,11.67,11.66,11.65,2600,5700,12800,11900,15800,11.73,11.74,11.75,11.76,11.78,8700,4800,1300,1000,1000,12.05,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544126,14323912.0,1.597216e+15,243803346.0,23.0,601086,150001000.0,44329073,3.063139e+08,6.91,6.91,6.90,6.89,6.88,6.87,18352127,57000,700,13000,9100,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,6.91,NaN,NaN,NaN,NaN,NaN,NaN
1544171,14324134.0,1.597216e+15,243803568.0,23.0,600209,150001000.0,6528000,3.583416e+07,5.45,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,5.45,5.46,5.47,5.48,5.49,1032711,41100,23100,31800,4000,5.74,NaN,NaN,NaN,NaN,NaN,NaN
1544212,14324262.0,1.597216e+15,243803696.0,23.0,688180,150001000.0,4005997,4.479805e+08,112.90,112.94,0.00,0.00,0.00,0.00,40037,0,0,0,0,112.94,0.00,0.00,0.00,0.00,40037,17820,0,0,0,115.00,NaN,NaN,NaN,NaN,NaN,NaN
1544227,14324280.0,1.597216e+15,243803714.0,23.0,601399,150001000.0,34436793,1.829526e+08,5.45,5.44,5.43,5.42,5.41,5.40,213860,65300,37900,65400,34200,5.45,0.00,0.00,0.00,0.00,803511,0,0,0,0,5.13,NaN,NaN,NaN,NaN,NaN,NaN


0.04910797824157139

In [10]:
pd.set_option('max_columns', 200)
test[np.isnan(test["sequenceNo_x"]) & (test['time_y'] >= 145700000) & (test['bid2p'] != 0)]

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange,time_y
3121773,NaN,NaN,NaN,NaN,600742,NaN,11444639,1.361709e+08,12.00,12.00,11.99,11.97,11.94,11.92,8028285,127300,800,14500,2300,12.00,0.0,0.000,0.000,0.00,479900,0,0,0,0,11.42,3506329.0,1.597302e+15,51403162.0,25.0,SH,150000000.0
3121774,NaN,NaN,NaN,NaN,600939,NaN,31651374,1.411801e+08,4.70,4.70,4.69,4.68,4.67,4.66,16428400,256800,345900,68600,1000,4.70,0.0,0.000,0.000,0.00,2224300,0,0,0,0,4.20,3506353.0,1.597302e+15,51405279.0,25.0,SH,150000000.0
3121775,NaN,NaN,NaN,NaN,600157,NaN,296667430,4.476988e+08,1.55,1.55,1.54,1.53,1.52,1.51,60447660,648900,615800,709900,1050900,1.55,0.0,0.000,0.000,0.00,2501600,0,0,0,0,1.41,3506362.0,1.597302e+15,51405288.0,25.0,SH,150000000.0
3121966,NaN,NaN,NaN,NaN,600313,NaN,131655277,6.378331e+08,5.02,5.02,5.01,5.00,4.99,4.98,23598218,123000,103700,28200,61600,5.02,0.0,0.000,0.000,0.00,39700,0,0,0,0,4.52,3506615.0,1.597302e+15,51405541.0,25.0,SH,150000000.0
3121967,NaN,NaN,NaN,NaN,601456,NaN,2538963,3.661185e+07,14.42,14.42,14.41,14.40,14.39,14.38,55158977,36900,43800,10400,7200,14.42,0.0,0.000,0.000,0.00,78670,0,0,0,0,14.42,3506620.0,1.597302e+15,51405546.0,25.0,SH,150000000.0
3121968,NaN,NaN,NaN,NaN,600166,NaN,182335824,5.255778e+08,2.95,2.95,2.94,2.93,2.92,2.91,42660700,4670300,582300,285200,336900,2.95,0.0,0.000,0.000,0.00,808600,0,0,0,0,2.80,3506622.0,1.597302e+15,51405548.0,25.0,SH,150000000.0
3121970,NaN,NaN,NaN,NaN,603809,NaN,6200112,1.584826e+08,25.96,25.96,25.95,25.90,25.88,25.81,502991,4300,3900,200,3000,25.96,0.0,0.000,0.000,0.00,4800,0,0,0,0,24.00,3506658.0,1.597302e+15,51413668.0,25.0,SH,150000000.0
3121971,NaN,NaN,NaN,NaN,603077,NaN,579469263,8.758387e+08,1.55,1.55,1.54,1.53,1.52,1.51,9093474,1664990,4184399,4256200,2198300,1.55,0.0,0.000,0.000,0.00,3304826,0,0,0,0,1.42,3506686.0,1.597302e+15,51415364.0,25.0,SH,150000000.0
3121972,NaN,NaN,NaN,NaN,601086,NaN,2711329,2.060610e+07,7.60,7.60,7.59,7.58,7.56,7.55,14518500,15700,21600,5400,2800,7.60,0.0,0.000,0.000,0.00,34200,0,0,0,0,7.60,3506690.0,1.597302e+15,51415368.0,25.0,SH,150000000.0
3122049,NaN,NaN,NaN,NaN,600071,NaN,22185509,3.036791e+08,14.19,14.19,14.18,14.16,14.08,14.07,1051192,3900,100,1000,100,14.19,0.0,0.000,0.000,0.00,162100,0,0,0,0,12.83,3507113.0,1.597302e+15,51442333.0,25.0,SH,150001000.0


In [12]:
pd.set_option('max_columns', 200)
test[np.isnan(test["sequenceNo_x"]) & (test['openPrice'] != 0)]

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange,time_y
2208159,NaN,NaN,NaN,NaN,603555,NaN,140900,2.733460e+05,1.94,1.94,1.93,1.90,1.88,1.85,3866500,16000,10000,30000,600,1.940,0.00,0.00,0.000,0.00,140900,0,0,0,0,1.94,156655.0,1.597282e+15,1740442.0,25.0,SH,92502000.0
2208160,NaN,NaN,NaN,NaN,601086,NaN,932075,7.083770e+06,7.60,7.60,7.59,7.58,7.57,7.55,18479625,3700,6800,1300,3500,7.600,0.00,0.00,0.000,0.00,932075,0,0,0,0,7.60,156714.0,1.597282e+15,1740501.0,25.0,SH,92502000.0
2208161,NaN,NaN,NaN,NaN,601456,NaN,711570,1.026084e+07,14.42,14.42,14.41,14.40,14.39,14.38,693558730,62400,109600,4900,4100,14.420,0.00,0.00,0.000,0.00,705800,0,0,0,0,14.42,156769.0,1.597282e+15,1741500.0,25.0,SH,92502000.0
2208162,NaN,NaN,NaN,NaN,600086,NaN,1134400,1.349936e+06,1.19,1.19,0.00,0.00,0.00,0.00,3226500,3223400,0,0,0,1.200,1.21,1.22,1.240,1.25,400,5300,11400,1600,52400,1.19,156804.0,1.597282e+15,1793578.0,25.0,SH,92502000.0
2208163,NaN,NaN,NaN,NaN,600234,NaN,900,1.007100e+04,11.19,11.19,11.16,11.15,11.11,11.10,1722200,2000,1500,1500,1000,11.190,0.00,0.00,0.000,0.00,900,0,0,0,0,11.19,156871.0,1.597282e+15,1793645.0,25.0,SH,92502000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214364,NaN,NaN,NaN,NaN,601086,NaN,2479829,1.884670e+07,7.60,7.60,7.59,7.58,7.56,7.55,16114200,22200,28800,9700,5500,7.600,0.00,0.00,0.000,0.00,932075,0,0,0,0,7.60,2457012.0,1.597287e+15,60009378.0,25.0,SH,104711000.0
2214365,NaN,NaN,NaN,NaN,603167,NaN,56752413,8.094779e+08,14.53,14.53,14.52,14.51,14.50,14.49,3249341,14800,8600,5700,100,14.530,0.00,0.00,0.004,0.00,250600,0,0,200,0,12.95,2457684.0,1.597287e+15,60020801.0,25.0,SH,104712000.0
2214366,NaN,NaN,NaN,NaN,601456,NaN,2006582,2.893491e+07,14.42,14.42,14.41,14.40,14.39,14.38,307337558,37400,53400,7100,13000,14.420,0.00,0.00,0.000,0.00,705800,0,0,0,0,14.42,2457805.0,1.597287e+15,60023202.0,25.0,SH,104713000.0
2214367,NaN,NaN,NaN,NaN,600316,NaN,27786284,1.005368e+09,37.39,37.39,37.38,37.35,37.34,37.31,8196547,7200,5700,1300,3900,0.001,0.00,0.00,0.000,0.00,190,0,0,0,0,33.90,2457969.0,1.597287e+15,60025679.0,25.0,SH,104713000.0


In [153]:
data2_1[data2_1['time'] >= 145700000]

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
2978407,3438863,1597301820123452,50945158,25,600608,SH,145700000,1565219,6.998381e+06,4.48,4.47,4.46,4.45,4.44,4.43,25100,20200,10500,83900,16500,4.48,4.49,4.50,4.51,4.52,22400,14700,56200,42000,25400,4.45
2978408,3438864,1597301820141784,50945203,25,600050,SH,145700000,476045508,2.520347e+09,5.31,0.00,0.00,5.28,5.27,5.26,0,0,1697900,1002200,1183200,0.00,0.00,5.33,5.34,5.35,0,0,4685700,6131238,12586653,5.23
2978409,3438865,1597301820141790,50945204,25,600130,SH,145700000,22896435,1.022036e+08,4.47,0.00,0.00,4.44,4.43,4.42,0,0,264400,70600,180700,0.00,0.00,4.49,4.50,4.51,0,0,77400,336100,99700,4.45
2978410,3438866,1597301820141792,50945205,25,600511,SH,145700000,31861576,1.319297e+09,41.02,0.00,0.00,41.00,40.99,40.98,0,0,15200,13700,47121,0.00,0.00,41.05,41.06,41.07,0,0,10000,6500,2200,41.76
2978411,3438867,1597301820141795,50945206,25,600601,SH,145700000,49939897,2.743644e+08,5.40,0.00,0.00,5.37,5.36,5.35,0,0,241100,628400,294300,0.00,0.00,5.42,5.43,5.44,0,0,118410,224600,17500,5.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042987,3508507,1597302002839030,51499294,25,688222,SH,150002000,829869,3.671412e+07,44.30,44.20,44.17,44.16,44.15,44.12,3819,2667,646,900,1500,44.30,44.31,44.32,44.33,44.34,9564,500,766,3000,2000,44.18
3042988,3508508,1597302002839033,51499295,25,688360,SH,150002000,1783684,8.939384e+07,50.28,50.27,50.26,50.25,50.24,50.23,453,3400,11920,2065,700,50.28,50.30,50.31,50.35,50.39,1407,725,3000,1200,6000,48.48
3042989,3508509,1597302002839036,51499296,25,688520,SH,150002000,1670243,1.197255e+08,71.12,71.12,71.10,71.09,71.08,71.07,4069,200,500,3128,1200,71.13,71.14,71.26,71.28,71.40,2500,2700,9700,300,400,72.00
3042990,3508510,1597302002839037,51499297,25,688050,SH,150002000,1277221,3.043572e+08,230.00,229.95,229.60,229.21,229.00,228.99,362,200,352,1000,200,230.00,230.35,230.48,230.49,230.50,86,5200,400,2800,200,243.69


In [112]:
for i in test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] == 112237000.0)]['StockID'].unique():
    try:
        assert(data2_1[data2_1['StockID'] == i]['time'].max() < 112237000)
    except:
        print(data2_1[(data2_1['StockID'] == i) & (data2_1['time'] >= 112237000)])
        print(data1_1[(data1_1['StockID'] == i) & (data1_1['time'] >= 112237000)])

In [127]:
for i in test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] == 112257000.0)]['StockID'].unique():
    try:
        assert(data2_1[data2_1['StockID'] == i]['time'].min() > 112257000)
    except:
        print(data1_1[(data1_1['StockID'] == i) & (data1_1['time'] <= 112257000)])
        print(data2_1[(data2_1['StockID'] == i) & (data2_1['time'] <= 112257000)])

       index    clockAtArrival  sequenceNo  source  StockID       time  \
203  6788438  1597289002199182   120797039      23   600083  112257000   

     cum_volume   cum_amount  close  bid1p  bid2p  bid3p  bid4p  bid5p  \
203     6297003  33431703.17   5.42   5.42   5.41    5.4   5.39   5.38   

       bid1q  bid2q  bid3q  bid4q  bid5q  ask1p  ask2p  ask3p  ask4p  ask5p  \
203  1753700  98800  15800   6000  48100    0.0    0.0    0.0    0.0    0.0   

     ask1q  ask2q  ask3q  ask4q  ask5q  openPrice  
203      0      0      0      0      0       5.15  
    index    clockAtArrival  sequenceNo  source  StockID exchange       time  \
88     88  1597288977903827        1955      25   600083       SH  112257000   

    cum_volume   cum_amount  close  bid1p  bid2p  bid3p  bid4p  bid5p  \
88     6297003  33431703.17   5.42   5.42   5.41    5.4   5.39   5.38   

      bid1q  bid2q  bid3q  bid4q  bid5q  ask1p  ask2p  ask3p  ask4p  ask5p  \
88  1753700  98800  15800   6000  48100   5.42    0.0

In [76]:
test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] != 104714000.0)].groupby('StockID')['time_x'].size().sort_values()

StockID
688600       1
688003       1
603991       1
603365       1
688081       1
603086       1
601952       1
688228       1
688369       1
600857       1
688021       1
600619       1
600665       1
688028       1
600508       1
600327       1
600530       1
603779       3
603818       3
600086       3
600255       7
603085       8
600416      16
600351      22
600083      23
600165      26
603555     107
600234     133
600634     187
600316     285
600078    1092
603167    1095
601086    1238
601456    1505
Name: time_x, dtype: int64

In [82]:
pd.merge(data2_1, test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] != 104714000.0)], on=["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q"]).shape[0]

16

In [80]:
test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] != 104714000.0) & (test['StockID'] == 601952)]

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange,time_y
1047638,2435776.0,1.597284e+15,56945693.0,23.0,601952,100214000.0,47256472,6.881190e+08,14.91,14.91,14.89,14.88,14.86,14.85,1700,175100,74840,54100,31200,14.92,14.93,0.0,0.0,0.0,131071,2017845,0,0,0,13.61,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
data2_1[(data2_1['StockID'] == 601952) & (data2_1['time'] >= 100200000.0)].head(10)

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
1042066,1176197,1597284122297450,36883431,25,601952,SH,100202000,47040792,6.849029e+08,14.90,14.90,14.89,14.88,14.86,14.85,16580,175900,76640,54100,27200,14.91,14.92,14.93,0.00,0.0,28500,227551,1857545,0,0,13.61
1043440,1177685,1597284125291115,36914823,25,601952,SH,100205000,47080272,6.854914e+08,14.90,14.89,14.88,14.86,14.85,14.84,175900,75240,54100,27200,122600,14.90,14.91,14.92,14.93,0.0,920,10400,229051,2003745,0,13.61
1044864,1179198,1597284128324882,36939542,25,601952,SH,100208000,47196272,6.872217e+08,14.92,14.89,14.88,14.86,14.85,14.84,177100,75240,54100,27200,122600,14.91,14.92,14.93,0.00,0.0,1000,141471,2004545,0,0,13.61
1046238,1180670,1597284131292707,36984947,25,601952,SH,100211000,47212872,6.874692e+08,14.90,14.89,14.88,14.86,14.85,14.84,176000,74840,54100,31200,122600,14.90,14.91,14.92,14.93,0.0,14000,2800,135171,2007145,0,13.61
1047632,1182169,1597284134304432,37018660,25,601952,SH,100214000,47256472,6.881190e+08,14.91,14.91,14.89,14.88,14.86,14.85,1700,175100,74840,54100,31200,14.92,14.93,14.92,14.93,0.0,131071,2017845,135171,2007145,0,13.61
1049041,1183678,1597284137295514,37050952,25,601952,SH,100217000,47269172,6.883083e+08,14.91,14.90,14.89,14.88,14.86,14.85,500,173600,74840,54100,31200,14.91,14.92,14.93,0.00,0.0,8500,188971,2017345,0,0,13.61
1050456,1185191,1597284140291853,37082743,25,601952,SH,100220000,47386572,6.900596e+08,14.90,14.90,14.89,14.88,14.86,14.85,10100,173300,74840,54100,31200,14.92,14.93,0.00,0.00,0.0,96971,2010345,0,0,0,13.61
1051819,1186670,1597284143288354,37113575,25,601952,SH,100223000,47414972,6.904827e+08,14.90,14.90,14.89,14.88,14.86,14.85,200,164000,74840,54100,31200,14.91,14.92,14.93,0.00,0.0,5500,101671,2019245,0,0,13.61
1053196,1188169,1597284146290612,37145130,25,601952,SH,100226000,47459372,6.911451e+08,14.90,14.90,14.89,14.88,14.86,14.85,60900,164000,76740,54100,31200,14.91,14.92,14.93,0.00,0.0,5600,66600,2039245,0,0,13.61
1054577,1189670,1597284149292594,37175893,25,601952,SH,100229000,47554172,6.925581e+08,14.91,14.91,14.90,14.89,14.88,14.86,4400,15700,164000,70840,54100,14.92,14.93,0.00,0.00,0.0,56600,2053345,0,0,0,13.61


In [77]:
test[np.isnan(test["sequenceNo_y"]) & (test['time_x'] != 104714000.0)]['StockID'].nunique()

34

In [71]:
data2_1[(data2_1['StockID'] == 600351) & (data2_1['time'] >= 93038000)][['StockID', 'time',
                                                                                  'cum_volume', 'cum_amount', 'close', 
           "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
           "bid4q", "bid5q", "ask1p", "ask2p", "ask1q", "ask2q"]].head(10)

,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask1q,ask2q
109133,600351,93038000,18270300,1.400939e+08,7.41,7.41,7.40,7.39,7.38,7.37,87800,311600,138800,593000,3039300,7.42,7.43,1500,1500
110819,600351,93041000,19784400,1.512817e+08,7.37,7.37,7.38,0.00,0.00,0.00,2799800,580700,0,0,0,7.39,7.45,1200,48800
112419,600351,93044000,21470300,1.637096e+08,7.37,7.37,7.38,0.00,0.00,0.00,1141900,580700,0,0,0,7.39,7.40,117400,10100
114117,600351,93047000,22236100,1.693644e+08,7.41,7.41,7.39,7.37,7.38,0.00,90500,4400,830500,580700,0,7.42,7.43,65100,24700
115686,600351,93050000,23414000,1.780525e+08,7.37,7.38,0.00,0.00,0.00,0.00,580700,0,0,0,0,7.37,7.38,422221,800
117316,600351,93053000,24006900,1.824221e+08,7.37,7.38,0.00,0.00,0.00,0.00,580700,0,0,0,0,7.37,7.38,358121,389200
118921,600351,93056000,24717630,1.876603e+08,7.37,7.37,7.38,0.00,0.00,0.00,93470,580700,0,0,0,7.38,7.39,677300,7000
120605,600351,93059000,26204430,1.986199e+08,7.37,7.37,7.38,0.00,0.00,0.00,848370,580700,0,0,0,7.38,7.39,518084,7400
122236,600351,93102000,26641230,2.018410e+08,7.37,7.37,7.38,0.00,0.00,0.00,1027470,580700,0,0,0,7.38,7.39,322684,6600
123876,600351,93105000,27421714,2.075978e+08,7.38,7.38,7.37,7.38,0.00,0.00,806516,934070,580700,0,0,7.40,7.41,28700,28700


In [72]:
data1_1[(data1_1['StockID'] == 600351) & (data1_1['time'] >= 93038000)][['StockID', 'time',
                                                                                  'cum_volume', 'cum_amount', 'close', 
           "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
           "bid4q", "bid5q", "ask1p", "ask2p", "ask1q", "ask2q"]].head(10)

,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask1q,ask2q
109139,600351,93038000,18270300,1.400939e+08,7.41,7.41,7.40,7.39,7.38,7.37,87800,311600,138800,593000,3039300,7.42,7.43,1500,1500
110825,600351,93041000,19784400,1.512817e+08,7.37,7.37,0.00,0.00,0.00,0.00,2799800,0,0,0,0,7.39,7.45,1200,48800
112425,600351,93044000,21470300,1.637096e+08,7.37,7.37,0.00,0.00,0.00,0.00,1141900,0,0,0,0,7.39,7.40,117400,10100
114123,600351,93047000,22236100,1.693644e+08,7.41,7.41,7.39,7.37,0.00,0.00,90500,4400,830500,0,0,7.42,7.43,65100,24700
115692,600351,93050000,23414000,1.780525e+08,7.37,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,7.37,7.38,422221,800
117322,600351,93053000,24006900,1.824221e+08,7.37,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,7.37,7.38,358121,389200
118927,600351,93056000,24717630,1.876603e+08,7.37,7.37,0.00,0.00,0.00,0.00,93470,0,0,0,0,7.38,7.39,677300,7000
120611,600351,93059000,26204430,1.986199e+08,7.37,7.37,0.00,0.00,0.00,0.00,848370,0,0,0,0,7.38,7.39,518084,7400
122242,600351,93102000,26641230,2.018410e+08,7.37,7.37,0.00,0.00,0.00,0.00,1027470,0,0,0,0,7.38,7.39,322684,6600
123882,600351,93105000,27421714,2.075978e+08,7.38,7.38,7.37,0.00,0.00,0.00,806516,934070,0,0,0,7.40,7.41,28700,28700


In [39]:
logSH[(logSH['StockID'] == 603779) & (logSH['cum_volume'] == 253500)]

,clockAtArrival,sequenceNo,source,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
352030,1597281925036807,4239992,23,603779,92500000,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,0.00
355985,1597281928661013,4498605,22,603779,92504360,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
356120,1597281928671847,4498827,23,603779,92504000,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,0.00
357043,1597281931314762,4565738,23,603779,92506000,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
358311,1597281944234711,4684754,22,603779,92518770,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
360773,1597281973074224,4706399,22,603779,92548770,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
362348,1597281991288414,4737706,23,603779,92606000,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
363311,1597282003081411,4746008,22,603779,92618410,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
365786,1597282033089632,4767260,22,603779,92648510,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31
367361,1597282051290336,4798656,23,603779,92706000,253500,1599585.0,6.31,6.31,0.0,0.0,0.0,0.0,30200,0,0,0,0,6.33,6.34,6.35,6.38,6.4,39100,1000,800,10000,31900,6.31


In [85]:
data1_1[data1_1['StockID'] == 300]

,index,clockAtArrival,sequenceNo,source,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
1,358511,1597281945422947,4686974,22,300,92521140,1255841,2.067322e+09,4666.4901,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
5,389289,1597282230491286,5630735,22,300,93006190,2297389,3.515177e+09,4668.4877,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
9,394689,1597282235447340,6153260,22,300,93011160,2861903,4.596104e+09,4670.6107,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
13,400809,1597282240427708,6687511,22,300,93016140,3125459,4.996339e+09,4671.7109,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
17,406906,1597282245509278,7215209,22,300,93021220,3377510,5.436142e+09,4671.7689,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,4937827,1597286830537749,95816355,22,300,104646200,72402789,1.167683e+11,4645.1303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3689,4943002,1597286835567426,95884389,22,300,104651160,72468679,1.168851e+11,4644.7384,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3693,4947407,1597286840481239,95951070,22,300,104656140,72498481,1.169494e+11,4644.5219,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3697,4952029,1597286845567827,96021139,22,300,104701230,72563410,1.170476e+11,4644.2308,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901


In [86]:
data2_1[data2_1['StockID'] == 300]

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
1,158559,1597281921201445,1985976,25,300,SH,92521000,1255841,2.067322e+08,4666.4901,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
5,170137,1597282207769467,2252997,25,300,SH,93006000,2297389,3.515177e+08,4668.4877,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
9,173005,1597282211327395,2361670,25,300,SH,93011000,2861903,4.596104e+08,4670.6107,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
13,176196,1597282216298436,2513381,25,300,SH,93016000,3125459,4.996339e+08,4671.7109,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
17,178816,1597282221395959,2679090,25,300,SH,93021000,3377510,5.436142e+08,4671.7689,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3689,2447606,1597286811225393,59848222,25,300,SH,104651000,72468679,1.168851e+10,4644.7384,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3693,2449838,1597286816229978,59887713,25,300,SH,104656000,72498481,1.169494e+10,4644.5219,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3697,2452163,1597286821309702,59926073,25,300,SH,104701000,72563410,1.170476e+10,4644.2308,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901
3701,2454720,1597286826273678,59967247,25,300,SH,104706000,72619986,1.171372e+10,4643.7352,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901


In [12]:
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]
data2['time'].min()

112301000

In [13]:
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['time'] >= 112301000)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

# for cols in ["openPrice"]:
#     data2_1[cols] = (data2_1[cols]/1000000).round(4)    
for cols in ["cum_amount"]:
    data2_1[cols] = (data2_1[cols]*10).round(4) 
    data1_1[cols] = data1_1[cols].round(4) 
    
    
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])

5989
5988
5989


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,time_x,cum_volume,cum_amount,close,bid1p_x,bid2p_x,bid3p_x,bid4p_x,bid5p_x,bid1q_x,bid2q_x,bid3q_x,bid4q_x,bid5q_x,ask1p_x,ask2p_x,ask3p_x,ask4p_x,ask5p_x,ask1q_x,ask2q_x,ask3q_x,ask4q_x,ask5q_x,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange,time_y,bid1p_y,bid2p_y,bid3p_y,bid4p_y,bid5p_y,bid1q_y,bid2q_y,bid3q_y,bid4q_y,bid5q_y,ask1p_y,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y
5410,13066581,1597301155787830,206265189,22,300,144531140,143270582,2.350902e+11,4634.4614,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4666.4901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
import pandas as pd
import random
import numpy as np
import pickle 

TradeLog = pd.read_csv(r'F:\data\20200821\logs_20200821_zt_88_03_day_88data\mdTradeLog_20200821_0900.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "TransactTime", "SecurityID", "exchId", 
                                                 "ApplSeqNum", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]]
TradeLog1 = pd.read_csv(r'F:\data\20200821\full_20200821_zt_70_01_day_7079\mdTradeLog_20200821_0834.csv').loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog1 = TradeLog1[TradeLog1['exchId'] == 1]
TradeLog = TradeLog[TradeLog['exchId'] == 1]
display(TradeLog1.groupby('SecurityID')['TransactTime'].max().max())
display(TradeLog1.groupby('SecurityID')['TransactTime'].min().min())
display(TradeLog.groupby('SecurityID')['TransactTime'].max().max())
display(TradeLog.groupby('SecurityID')['TransactTime'].min().min())
# TradeLog = TradeLog[(TradeLog['TransactTime'] >= m_in)]
display(set(TradeLog['SecurityID'].unique()) - set(TradeLog1['SecurityID'].unique()))
display(set(TradeLog1['SecurityID'].unique()) - set(TradeLog['SecurityID'].unique()))
stocks = list(set(TradeLog1["SecurityID"].unique()) & set(TradeLog["SecurityID"].unique()))
display(len(stocks))
SH = TradeLog[TradeLog["SecurityID"].isin(stocks)]
SH1 = TradeLog1[TradeLog1["SecurityID"].isin(stocks)]
del TradeLog
del TradeLog1
print(len(SH["SecurityID"].unique()))
print(len(SH1["SecurityID"].unique()))


columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
# SH1['TradeQty'] = (SH1['TradeQty']/100).round(0)
# SH1['TradeMoney'] = (SH1['TradeMoney']/100).round(0)
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


150000920

92500000

150000920

92500000

set()

set()

1662

1662
1662
19834067
19834067
19834067
-----------------------------------------------


In [5]:
for i in re[np.isnan(re["sequenceNo_y"]) & (re['TransactTime'] == 105631350)]['SecurityID'].unique():
    try:
        assert(SH1[SH1['SecurityID'] == i]['TransactTime'].max() < 105631350)
    except:
        print(SH1[(SH1['SecurityID'] == i) & (SH1['TransactTime'] >= 105631350)].shape[0])
        print(SH[(SH['SecurityID'] == i) & (SH['TransactTime'] >= 105631350)].shape[0])

In [3]:
for i in re[np.isnan(re["sequenceNo_y"]) & (re['TransactTime'] == 105650110 )]['SecurityID'].unique():
    try:
        assert(SH1[SH1['SecurityID'] == i]['TransactTime'].min() > 105650110 )
    except:
        print(SH1[(SH1['SecurityID'] == i) & (SH1['TransactTime'] <= 105650110 )].shape[0])
        print(SH[(SH['SecurityID'] == i) & (SH['TransactTime'] <= 105650110 )].shape[0])

In [1]:
import pandas as pd
import random
import numpy as np
import pickle 

TradeLog = pd.read_csv(r'F:\data\20200821\logs_20200821_zs_96_03_day_96data\mdTradeLog_20200821_0859.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "TransactTime", "SecurityID", "exchId", 
                                                 "ApplSeqNum", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]]
TradeLog1 = pd.read_csv(r'F:\data\20200821\full_20200821_zt_70_01_day_7079\mdTradeLog_20200821_0834.csv').loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog1 = TradeLog1[TradeLog1['exchId'] == 2]
TradeLog = TradeLog[TradeLog['exchId'] == 2]
m_ax = TradeLog1.groupby('SecurityID')['TransactTime'].max().max()
m_in = TradeLog1.groupby('SecurityID')['TransactTime'].min().min()
display(m_ax)
display(m_in)
m_ax = TradeLog.groupby('SecurityID')['TransactTime'].max().max()
m_in = TradeLog.groupby('SecurityID')['TransactTime'].min().min()
display(m_ax)
display(m_in)
display(TradeLog1['TradeBSFlag'].unique())
display(TradeLog['TradeBSFlag'].unique())
display(TradeLog1['ExecType'].unique())
# TradeLog = TradeLog[(TradeLog['TransactTime'] >= m_in)]
display(set(TradeLog['SecurityID'].unique()) - set(TradeLog1['SecurityID'].unique()))
display(set(TradeLog1['SecurityID'].unique()) - set(TradeLog['SecurityID'].unique()))
stocks = list(set(TradeLog1["SecurityID"].unique()) & set(TradeLog["SecurityID"].unique()))
display(len(stocks))
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(stocks)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(stocks)]
del TradeLog
del TradeLog1
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
# TradeLogSZ1['TradeQty'] = (TradeLogSZ1['TradeQty']/100).round(0)
# TradeLogSZ1['TradeMoney'] = (TradeLogSZ1['TradeMoney']/100).round(0)
re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


150000000

91500030

150000000

91500030

...

array([4, '4', 'F'], dtype=object)

set()

set()

2233

2233
2233
39845059
39845059
39845059
-----------------------------------------------


In [2]:
for i in re[np.isnan(re["sequenceNo_y"]) & (re['TransactTime'] == 112257400  )]['SecurityID'].unique():
    try:
        assert(TradeLogSZ1[TradeLogSZ1['SecurityID'] == i]['TransactTime'].min() > 112257400  )
    except:
        print(TradeLogSZ1[(TradeLogSZ1['SecurityID'] == i) & (TradeLogSZ1['TransactTime'] <= 112257400  )].shape[0])
        print(TradeLogSZ[(TradeLogSZ['SecurityID'] == i) & (TradeLogSZ['TransactTime'] <= 112257400  )].shape[0])

In [1]:
import pandas as pd
import random
import numpy as np
import pickle 

OrderLog = pd.read_csv(r'F:\data\20200812\logs_20200812_zs_96_03_day_96data\mdOrderLog_20200812_0849.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "TransactTime", "exchId", 
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
OrderLog1 = pd.read_csv(r'F:\data\20200812\full_20200812_zt_70_01_day_7079\mdOrderLog_20200812_0840.csv').loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
m_ax = OrderLog1.groupby('SecurityID')['TransactTime'].max().max()
display(m_ax)
m_in = OrderLog1.groupby('SecurityID')['TransactTime'].min().min()
display(m_in)
display(OrderLog["OrderType"].unique())
display(OrderLog1["OrderType"].unique())

OrderLog = OrderLog[(OrderLog['TransactTime'] <= m_ax)]
display(set(OrderLog['SecurityID'].unique()) - set(OrderLog1['SecurityID'].unique()))
display(set(OrderLog1['SecurityID'].unique()) - set(OrderLog['SecurityID'].unique()))
stocks = list(set(OrderLog1["SecurityID"].unique()) & set(OrderLog["SecurityID"].unique()))
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(stocks)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(stocks)]
del OrderLog
del OrderLog1
OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
# OrderLogSZ1['OrderQty'] = (OrderLogSZ1['OrderQty']/100).round(0)
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


103618500

91500000

array([2, 1, '2', 'U', '1'], dtype=object)

array([2, 1, '2', 'U', '1'], dtype=object)

...

set()

2235
2235
26448695
26448640
26448695
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,TransactTime,exchId_x,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,exchId_y
121800,1597194909253810,510409,91500570,2,29012,2175,1,2,14700,17200,NaN,NaN,NaN
121801,1597194909253811,510410,91500570,2,29013,567,1,2,131200,100,NaN,NaN,NaN
121802,1597194909253814,510411,91500570,2,29014,2991,1,2,1196400,200,NaN,NaN,NaN
121803,1597194909253815,510412,91500570,2,29015,856,1,2,103700,1000,NaN,NaN,NaN
121804,1597194909253818,510413,91500570,2,29016,2847,1,2,1300000,100,NaN,NaN,NaN
121805,1597194909253818,510414,91500570,2,29017,612,1,2,75400,500,NaN,NaN,NaN
121806,1597194909253819,510415,91500570,2,29018,2991,1,2,1196400,500,NaN,NaN,NaN
121807,1597194909253820,510416,91500570,2,29019,30,2,2,86000,1000,NaN,NaN,NaN
121808,1597194909253820,510417,91500570,2,29020,753,2,2,31400,2000,NaN,NaN,NaN
121809,1597194909253821,510418,91500570,2,29021,2885,1,2,162600,100,NaN,NaN,NaN


55
[ 91500570 103618500]
45
[  2175    567   2991    856   2847    612     30    753   2885   2988
 300843   2475   2153   2099    650   2700    524 300750 300334 300021
 300086   2610   2625 300622   1896   2665    876 300011   2517   2413
   2074 300014   2503   2237   2172 300676   2550   2685      9   2436
   2959    625   2036   2382 300443]


In [3]:
for i in ree[np.isnan(ree["sequenceNo_y"]) & (ree['TransactTime'] == 103618500)]['SecurityID'].unique():
    try:
        assert(OrderLogSZ1[OrderLogSZ1['SecurityID'] == i]['TransactTime'].max() < 103618500)
    except:
        print(OrderLogSZ1[(OrderLogSZ1['SecurityID'] == i) & (OrderLogSZ1['TransactTime'] >= 103618500)].shape[0])
        print(OrderLogSZ[(OrderLogSZ['SecurityID'] == i) & (OrderLogSZ['TransactTime'] >= 103618500)].shape[0])

1
2


In [5]:
import pandas as pd
import csv
with open(r'F:\data\20200812\logs_20200812_zs_96_03_day_96data\mdOrderLog_20200812_0849.csv') as f:
    reader = csv.DictReader(f)    
    for row in reader:
        if row['OrderType'] not in {'1', '2', 'U'}:
            print(row)
with open(r'F:\data\20200812\logs_20200812_zs_96_03_day_96data\mdTradeLog_20200812_0849.csv') as f:
    reader = csv.DictReader(f)    
    for row in reader:
        if row['ExecType'] not in {'4', 'F'}:
            print(row)

In [4]:
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'F:\data\20200814\logs_20200814_zs_92_01_day_data\mdLog_SZ_20200814_0836.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "StockID", 'source', "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
logSZ1 = pd.read_csv(r'F:\data\20200814\full_20200814_zt_70_01_day_7079\mdLog_SZ_20200814_1153.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]

print("new data source type: ")
print(logSZ1["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())

logSZ["time"] = logSZ["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))
logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")))) * 1000

m_in = logSZ1['time'].min()
m_ax = logSZ1['time'].max()
print(m_in)
print(m_ax)

data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ['time'] >= m_in) & (logSZ['source'] == 4)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", 'openPrice']
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]

for cols in ["cum_amount"]:
    data2_1[cols] = data2_1[cols].round(2)
    data1_1[cols] = data1_1[cols].round(2)
    
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

new data source type: 
[25]
['08:37:03.000' '08:38:03.000' '08:39:03.000' ... '15:15:03.000'
 '15:16:03.000' '15:17:03.000']
['11:54:00' '11:54:03' '11:55:00' ... '15:29:03' '15:30:03' '15:31:03']
115400000
153103000
2236
2236
3756972
3756972
3756972
-----------------------------------------------
